In [1]:
import string
from datetime import datetime
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict

import nltk
# nltk.data.path.append("/")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\Eric\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import requests
requests.packages.urllib3.disable_warnings()

In [3]:
print("hi")

hi


In [4]:
reviews = []
with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        reviews.append(json_line)

In [5]:
print(len(reviews))
reviews[0]

4153150


{u'business_id': u'2aFiy99vNLklCx3T_tGS9A',
 u'cool': 0,
 u'date': u'2011-10-10',
 u'funny': 0,
 u'review_id': u'NxL8SIC5yqOdnlXCg18IBg',
 u'stars': 5,
 u'text': u"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.",
 u'type': u'review',
 u'useful': 0,
 u'user_id': u'KpkOkG6RIf4Ra25Lhhxf1A'}

In [6]:
businesses = []
with open("yelp_academic_dataset_business.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        businesses.append(json_line)

In [7]:
print(len(businesses))
businesses[0]

144072


{u'address': u'227 E Baseline Rd, Ste J2',
 u'attributes': [u'BikeParking: True',
  u'BusinessAcceptsBitcoin: False',
  u'BusinessAcceptsCreditCards: True',
  u"BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  u'DogsAllowed: False',
  u'RestaurantsPriceRange2: 2',
  u'WheelchairAccessible: True'],
 u'business_id': u'0DI8Dt2PJp07XkVvIElIcQ',
 u'categories': [u'Tobacco Shops', u'Nightlife', u'Vape Shops', u'Shopping'],
 u'city': u'Tempe',
 u'hours': [u'Monday 11:0-21:0',
  u'Tuesday 11:0-21:0',
  u'Wednesday 11:0-21:0',
  u'Thursday 11:0-21:0',
  u'Friday 11:0-22:0',
  u'Saturday 10:0-22:0',
  u'Sunday 11:0-18:0'],
 u'is_open': 0,
 u'latitude': 33.3782141,
 u'longitude': -111.936102,
 u'name': u'Innovative Vapors',
 u'neighborhood': u'',
 u'postal_code': u'85283',
 u'review_count': 17,
 u'stars': 4.5,
 u'state': u'AZ',
 u'type': u'business'}

In [9]:
valid_business_ids = set()
business_id_to_index = {}
business_index_to_bid = {}
c = 0
for business in businesses:
    if business["review_count"] >= 1000:
        valid_business_ids.add(business["business_id"])
        if business["business_id"] not in business_id_to_index:
            business_id_to_index[business["business_id"]] = c
            business_index_to_bid[c] = business["business_id"]
            c += 1
print(len(valid_business_ids))
valid_business_ids

177


{u'--9e1ONYQuAa-CB_Rrw7Tw',
 u'-ed0Yc9on37RoIoG2ZgxBA',
 u'0FUtlsQrJI7LhqDPxLumEw',
 u'0NmTwqYEQiKErDv4a55obg',
 u'0W4lkclzZThpx3V65bVgig',
 u'2iTsRqUsPGRH1li1WVRvKQ',
 u'2weQS-RnoOBhb1KsHKyoSQ',
 u'3BCsAgo_1i4xMuTyLKMLRQ',
 u'3GEEy7RP6e4bT4LAiWFMFQ',
 u'3N9U549Zse8UP-MwKZAjAQ',
 u'3kdSl5mo9dWC4clrQjEDGg',
 u'3l54GTr8-E3XPbIxnF_sAA',
 u'3xmfT7l3xNH5LK1dLzfvGw',
 u'4GXII-GU7S0ZyU6ElkhscQ',
 u'4JNXUYY8wbaaDmk3BPzlWw',
 u'4k3RlMAMd46DZ_JyZU0lMg',
 u'5LNZ67Yw9RD6nf4_UhXOjw',
 u'5T6kFKFycym_GkhgOiysIw',
 u'5iSmZO0SrKU6EoXK_1M8Kw',
 u'5shgJB7a-2_gdnzc0gsOtg',
 u'6Ct57qgmXwOnzfSZoUGh0Q',
 u'6Q7-wkCPc1KF75jZLOTcMw',
 u'7fxebHYUwIF6CakxSr70iQ',
 u'7sPNbCx7vGAaH7SbNPZ6oA',
 u'7sb2FYLS2sejZKxRYF9mtg',
 u'9SU7ZZhaFUJJ6m2k5HKHeg',
 u'9a3DrZvpYxVs3k_qwlCNSw',
 u'A-uZAD4zP3rRxb44WUGV5w',
 u'A5Rkh7UymKm0_Rxm9K2PJw',
 u'AV6weBrZFFBfRGCbcRGO4g',
 u'Az_60nNuh1FH8Ds8oasZjw',
 u'BH9z7IJ4zydAqgwsbqoVZQ',
 u'BIBWGO_r_1znnlmLbp4Nxg',
 u'BLIJ-p5wYuAhw6Pp6mh6mw',
 u'ByFMv3p5X1aNeZhU61rDcA',
 u'CiYLq33nAyghFkUR1

In [ ]:
only_text = []
ordered_review_ids = []
ordered_business_id_indices = []
bid_to_rid_dict = defaultdict(list)
rid_to_bid_dict = {}
for review in reviews:
    if review["business_id"] in valid_business_ids:
        only_text.append(review["text"])
        ordered_review_ids.append(review["review_id"])
        ordered_business_id_indices.append(business_id_to_index[review["business_id"]])
        bid_to_rid_dict[review["business_id"]].append(review["review_id"])
        rid_to_bid_dict[review["review_id"]] = review["business_id"]
print(len(only_text))
print(len(ordered_review_ids))

In [ ]:

tfidf_vectorizer = TfidfVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
tfidf_review_text = tfidf_vectorizer.fit_transform(only_text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print(tfidf_review_text.shape)

In [ ]:
count_vectorizer = CountVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
count_review_text = count_vectorizer.fit_transform(only_text)
count_feature_names = count_vectorizer.get_feature_names()

print(count_review_text.shape)

In [ ]:
print(count_feature_names)

In [ ]:
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [ ]:
lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_text)
display_topics(lda_model, count_feature_names, 20)

In [ ]:
nmf_model = NMF(n_components=30).fit(tfidf_review_text)
display_topics(nmf_model, tfidf_feature_names, 20)

In [ ]:
tagged_doc_list = []
for i in range(len(only_text)):
    text = only_text[i]
    tokens = nltk.word_tokenize(text)
    label = ordered_business_id_indices[i]
    tagged_doc = TaggedDocument(tokens, [label])
    tagged_doc_list.append(tagged_doc)

In [ ]:
tagged_doc_list[0]

In [ ]:
only_text[0]

In [ ]:
model = Doc2Vec(size=500, min_count=100, iter=5)
model.build_vocab(tagged_doc_list)
%time model.train(tagged_doc_list)

In [ ]:
model.save('reviews_doc2vec_model.doc2vec')


In [ ]:
# load the model back
model_loaded = Doc2Vec.load('reviews_doc2vec_model.doc2vec')

In [ ]:
example_index = 100

In [ ]:
example_bid = rid_to_bid_dict[ordered_review_ids[example_index]]
example_rids_for_business = bid_to_rid_dict[example_bid]

In [ ]:
print(len(example_rids_for_business))
example_bid

In [ ]:

print("Review Text is:")
print(only_text[example_index])

for rid in example_rids_for_business[:5]:
    ind = ordered_review_ids.index(rid)
    print
    print("---------------")
    print("Review index %d is:" % ind)
    print(only_text[ind])

In [ ]:
inferred = model.infer_vector(nltk.word_tokenize(only_text[100]))
inferred

In [ ]:
sims = model.docvecs.most_similar([inferred], topn=len(model.docvecs))
sims

In [ ]:
def compute_similarities(start_ind, end_ind):
    distances = []
    c = 0.0
    sum_vectors = np.zeros(500)
    for i in range(start_ind, end_ind+1):
        
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(1.0, c)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        sum_vectors += word2vec_vector
        c += 1
    return distances

In [ ]:
bid_to_min_ind = {}
bid_to_max_ind = {}
for i, business_index in enumerate(ordered_business_id_indices):
    if business_index_to_bid[business_index] not in bid_to_min_ind:
        bid_to_min_ind[business_index_to_bid[business_index]] = i
    bid_to_max_ind[business_index_to_bid[business_index]] = i


In [ ]:
example_bid_index = 1
example_bid = business_index_to_bid[example_bid_index]

In [ ]:
similarities = compute_similarities(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid])
similarities

In [ ]:
# Create a trace
trace = Scatter(
    x = range(len(similarities)),
    y = similarities
)

data = [trace]

py.iplot(data)